In [20]:

import torch
from transformers import LlamaForCausalLM, AutoModelForCausalLM,AutoTokenizer, LlamaTokenizer, LlamaTokenizerFast
import json
model_name_or_path = "/local1/ponienkung/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if isinstance(tokenizer, LlamaTokenizer) or isinstance(tokenizer, LlamaTokenizerFast):
    num_added_tokens = tokenizer.add_special_tokens({
        "bos_token": "<s>",
        "eos_token": "</s>",
        "unk_token": "<unk>",
        "pad_token": "<pad>",
    })
    assert num_added_tokens in [0, 1], "LlamaTokenizer should only add one special token - the pad_token, or no tokens if pad token present."


cnt_error = 0
pos = 0
neg = 0
seq_len = []
with open("../data/generated_data/train_800_10definitions_v2.json", "r") as F:
    for line in F.readlines():
        dat = json.loads(line)
        sentence = dat['prompt'].split("SENTENCE:")[1].split("EVENT TYPE")[0].strip()
        trigger = dat['completion'].replace("Event trigger is ", "").replace(".", "").strip()
        if trigger != "<trigger>":
            pos +=1
            if not trigger in sentence:
                cnt_error += 1
            seq_len.append(len(tokenizer(dat["prompt"] + dat["completion"]).input_ids))   
            if seq_len[-1] > 256:
                print(dat["prompt"])
        else:
            neg +=1
        
        
print(cnt_error, pos, neg)

SENTENCE: The government announced the elimination of the death penalty in their country. 
 EVENT TYPE: abolition. 
 DEFINITION: Abolition is a movement grounded in the belief that the status quo is not inevitable and that collective action can transform society, including the creation of new forms of power and decision-making outside of existing structures of oppression. 
 PARENT: action, SON: abolition, activity, adaptation, aggression, arranging, assignment, attempt, authorial_decision, countermeasure, debut, defense, deposition, detection, dieting, dribbling, drinking, editorial_decision, entrance, error, exit, expropriation, faux_pas, filling, grasping, indirection, insertion, interaction, looking, marking, means, nationalization, procreation, profiteering, punishment, pushing, reaction, repudiation, rubbing, safety_precaution, saving, scandalize, security, stacking, subdivision, tailing, veto, violence, voluntarism. 
 So what is the trigger?
SENTENCE: The university removed all g

In [18]:
import numpy as np
np.sum(np.array(seq_len) > 512)

164

In [66]:
data = json.load(open("data/generated_data_fix_v4.json", 'r'))

In [69]:
# print(data.keys())
ontology_size = []
events = {}
total_events = []
to_remove = []
for parent in data:
    ontology_size.append(len(data[parent]['events']))
    for ev in data[parent]['events']:
        if ev in events:
            events[ev].append([parent, ontology_size[-1]])
        else:
            events[ev] = [[parent, ontology_size[-1]]]
    total_events += data[parent]['events']
    # if ontology_size[-1] > 5:
    #     print(parent, data[parent]['events'])
print(len(total_events), len(set(total_events)))
ontology_size = np.array(ontology_size)
print(len(ontology_size))
print(np.sum(ontology_size > 5))

for ev, parents in events.items():
    if len(parents) > 1:
        print(ev, parents)

3917 3172
1693
62
dethronement [['abdication', 2], ['regime_change', 4]]
abrasion [['abrasion', 1], ['wear', 2], ['wound', 5]]
misconduct [['abuse', 4], ['anti-social_behaviour', 3], ['misconduct', 2]]
academic_dishonesty [['academic_dishonesty', 2], ['cheating', 4]]
access [['access', 1], ['means', 4], ['rights', 4]]
transport_accident [['accident', 6], ['transport_accident', 4]]
distress [['accident', 6], ['psychological_stress', 2]]
electrical_accident [['accident', 6], ['electrical_accident', 1]]
water_accident [['accident', 6], ['water_accident', 2]]
accident [['accident', 6], ['incident', 6]]
accommodation [['accommodation', 1], ['human-centered_design', 2]]
credit [['acknowledgment', 2], ['credit', 2], ['loan', 2]]
acknowledgment [['acknowledgment', 2], ['assertion', 4], ['thanking', 2]]
sound [['acoustic_wave', 2], ['sound', 4]]
acoustic_wave [['acoustic_wave', 2], ['elastic_wave', 2]]
enrollment [['acquisition', 5], ['enrollment', 2]]
loan [['acquisition', 5], ['financial_prod